In [1]:
from icecream import ic
import torch

In [2]:
# get txt
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
ic(vocab_size)

ic| vocab_size: 65


65

In [4]:
# letter tokens, cuz easy. maybe improve later.

# map chars to ints

# create a dict to map all chars to ints
ch_to_int = {ch:i for i, ch in enumerate(chars)}
# and vice versa
int_to_ch = {i:ch for i, ch in enumerate(chars)}
# didn't know that list comprehension also worked for dicts, very cool!

#def encode & decode functions
encode = lambda s: [ch_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_ch[i] for i in l])

ic(decode(encode("lezz go")))

ic| decode(encode("lezz go")): 'lezz go'


'lezz go'

In [5]:
# encode text and store as torch
data = torch.tensor(encode(text), dtype = torch.long)
ic(data.shape,data.dtype)
ic(data[:100])

ic| data.shape: torch.Size([1115394]), data.dtype: torch.int64
ic| data[:100]: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
                        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
                         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
                        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
                         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
                        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [6]:
# split training and validation set.

# use last 10th of data as validation. smarter options are availible
t_ratio = int(0.9*len(data))
train_data = data[:t_ratio]
val_data = data[t_ratio:]

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [13]:
torch.manual_seed(1)
batch_size = 4
block_size = 8

def get_batch(split):
    # training or validating?
    data = train_data if split == 'train' else val_data
    # pick #batch_size random starting points in data
    rand_starts = torch.randint(len(data)- block_size, (batch_size,))
    # inputs
    inputs = torch.stack([data[i:i+block_size] for i in rand_starts])
    expected_outputs = torch.stack([data[i+1:i+block_size+1] for i in rand_starts])
    return inputs, expected_outputs

ic(get_batch('train'))

ic| get_batch('train'): (tensor([[ 1, 51, 63,  1, 51, 39, 57, 58],
                                [42,  1, 58, 59, 56, 52,  1, 58],
                                [60, 47, 52, 43, 63, 39, 56, 42],
                                [ 0,  0, 25, 13, 30, 21, 13, 26]]),
                         tensor([[51, 63,  1, 51, 39, 57, 58, 43],
                                [ 1, 58, 59, 56, 52,  1, 58, 46],
                                [47, 52, 43, 63, 39, 56, 42,  1],
                                [ 0, 25, 13, 30, 21, 13, 26, 13]]))


(tensor([[ 1, 51, 63,  1, 51, 39, 57, 58],
         [42,  1, 58, 59, 56, 52,  1, 58],
         [60, 47, 52, 43, 63, 39, 56, 42],
         [ 0,  0, 25, 13, 30, 21, 13, 26]]),
 tensor([[51, 63,  1, 51, 39, 57, 58, 43],
         [ 1, 58, 59, 56, 52,  1, 58, 46],
         [47, 52, 43, 63, 39, 56, 42,  1],
         [ 0, 25, 13, 30, 21, 13, 26, 13]]))